In [2]:
# install pynetlogo
pip install pynetlogo

  Created wheel for pynetlogo: filename=pyNetLogo-0.4.1-py3-none-any.whl size=64521 sha256=3cfae6e79c5dc7da2f7d8febc9eb15f9373e44e67d0a91418f5d6bc6e8c69032
  Stored in directory: c:\users\frac1\appdata\local\pip\cache\wheels\08\8a\9c\3cdf0124bed0bb4876ae3fc4270ff5336b49b7ab081fb95072
Successfully built pynetlogo
Note: you may need to restart the kernel to use updated packages.


In [3]:
# install Java connection to python
pip install JPype1

Note: you may need to restart the kernel to use updated packages.


In [3]:
# import all necessary python libraries and the pyNetLogo library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')
import pyNetLogo

from SALib.sample import latin
from SALib.analyze import sobol

In [4]:
# set up the model
netlogo = pyNetLogo.NetLogoLink(gui=False) #Mac users should set this to False

netlogo.load_model('./Model.nlogo')

In [50]:
problem = {
  'num_vars': 7,
  'names': ['administrative-network-meetings',
           'total-project-proposal-frequency',
           'informal-meetings-frequency',
           'green-energy-openness-change',
           'political-variety-change',
           'max-project-capacity',
           'random-intial-trust'],
  'bounds': [[0, 25],
            [0, 25],
            [0, 25],
            [-5,5],
            [-5,5],
            [0, 25],
            [0, 1]]
}
n = 10

param_values = np.round(latin.sample(problem, n))
param_values

array([[19., 14.,  0.,  4.,  1.,  5.,  1.],
       [ 2.,  3., 13., -5., -4.,  9.,  1.],
       [ 4., 22., 17., -0., -0., 14.,  0.],
       [20., 16.,  3.,  2., -5.,  1.,  1.],
       [24.,  1., 25.,  1.,  3., 23.,  0.],
       [17., 23., 11.,  4., -2., 12.,  0.],
       [14.,  9.,  7., -1.,  2.,  6.,  0.],
       [ 9., 19., 22., -3.,  1., 19.,  0.],
       [11.,  7., 19., -3.,  4., 16.,  1.],
       [ 6., 12.,  8.,  1., -1., 21.,  1.]])

In [51]:
df = pd.DataFrame(param_values)
df.iloc[:,-1] = df.iloc[:,-1].astype("bool")
param_values = np.array(df)
param_values

array([[19.0, 14.0, 0.0, 4.0, 1.0, 5.0, True],
       [2.0, 3.0, 13.0, -5.0, -4.0, 9.0, True],
       [4.0, 22.0, 17.0, -0.0, -0.0, 14.0, False],
       [20.0, 16.0, 3.0, 2.0, -5.0, 1.0, True],
       [24.0, 1.0, 25.0, 1.0, 3.0, 23.0, False],
       [17.0, 23.0, 11.0, 4.0, -2.0, 12.0, False],
       [14.0, 9.0, 7.0, -1.0, 2.0, 6.0, False],
       [9.0, 19.0, 22.0, -3.0, 1.0, 19.0, False],
       [11.0, 7.0, 19.0, -3.0, 4.0, 16.0, True],
       [6.0, 12.0, 8.0, 1.0, -1.0, 21.0, True]], dtype=object)

In [79]:
vector = []


[0, 1, 2, 3]

In [85]:
results = pd.DataFrame(index= vector, columns=['MW_implemented_wind', 'MW_implemented_solar', 'MW_implemented_urban'])
# for rs in range(5):
#     results.index
results

,MW_implemented_wind,MW_implemented_solar,MW_implemented_urban
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN


In [ ]:
results = pd.DataFrame(index= ['Random_seed'], columns=['MW_implemented_wind', 'MW_implemented_solar', 'MW_implemented_urban'])

for rs in range(5):
    
    netlogo.command('setup')
    netlogo.command('random-seed {}'.format(rs))
    results.loc["Random_seed"] = rs
    
    for run in range(param_values.shape[0]):       
        
        # Set the input parameters
        for i, name in enumerate(problem['names']):
            # Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(name, param_values[run,i]))


        # Run for 100 ticks and return the number of sheep and wolf agents at 
        # each time step
        MW_implemented = netlogo.repeat_report(['current-wind-production','current-solar-production', 'current-urban-production'], 100)
    
        # For each run, save the value of MW_implemented
        results.loc[run, 'MW_implemented_wind'] = MW_implemented['current-wind-production'].max()
        results.loc[run, 'MW_implemented_solar'] = MW_implemented['current-solar-production'].max()
        results.loc[run, 'MW_implemented_urban'] = MW_implemented['current-urban-production'].max()

In [58]:
results = pd.DataFrame(columns=['MW_implemented_wind', 'MW_implemented_solar', 'MW_implemented_urban'])

for run in range(param_values.shape[0]):
    for rs in range(5):
        netlogo.command('setup')
        netlogo.command('random-seed {}'.format(rs))
        
        # Set the input parameters
        for i, name in enumerate(problem['names']):
            # Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(name, param_values[run,i]))


        # Run for 100 ticks and return the number of sheep and wolf agents at 
        # each time step
        MW_implemented = netlogo.repeat_report(['current-wind-production','current-solar-production', 'current-urban-production'], 100)
    
        # For each run, save the value of MW_implemented
        results.loc[run, 'MW_implemented_wind'] = MW_implemented['current-wind-production'].max()
        results.loc[run, 'MW_implemented_solar'] = MW_implemented['current-solar-production'].max()
        results.loc[run, 'MW_implemented_urban'] = MW_implemented['current-urban-production'].max()

In [59]:
results

,MW_implemented_wind,MW_implemented_solar,MW_implemented_urban
Random_seed,NaN,NaN,NaN
0,15,75,69
1,5,15,7.5
2,49,193,111
3,30,45,106.5
4,0,0,0
5,35,179,121.5
6,0,45,51
7,30,90,93
8,20,15,21


In [ ]:
#Set the input parameters
for i, name in enumerate(problem['names']):
    if name == 'random-seed':
        #The NetLogo random seed requires a different syntax
        netlogo.command('random-seed {}'.format(i))
    else:
        #Otherwise, assume the input parameters are global variables
        netlogo.command('set {0} {1}'.format(name, experiment[i]))

netlogo.command('setup')
#Run for 100 ticks and return the number of sheep and wolf agents at each time step
MW_implemented = netlogo.repeat_report(['current-renewable-production "wind"','current-renewable-production "solar"', 'current-renewable-production "urban"'], 100)

In [19]:
# set up a simulation function that takes a single experiment (i.e. a vector of input parameters)
# as an argument, and returns the outcomes of interest in a pandas Series

def simulation(experiment):

    #Set the input parameters
    for i, name in enumerate(problem['names']):
        if name == 'random-seed':
            #The NetLogo random seed requires a different syntax
            netlogo.command('random-seed {}'.format(experiment[i]))
        else:
            #Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(name, experiment[i]))

    netlogo.command('setup')
    #Run for 100 ticks and return the number of sheep and wolf agents at each time step
    MW_implemented = netlogo.repeat_report(['current-renewable-production "wind"','current-renewable-production "solar"', 'current-renewable-production "urban"'], 100)


    return MW_implemented

In [18]:
# to close the netlogo workspace
netlogo.kill_workspace()